# Browser Automation change.org

In [17]:
import os
import random
import time
import asyncio

from playwright.async_api import async_playwright, expect

In [19]:
os.makedirs('data/', exist_ok=True)

### 1) open browser, visit change.org

In [20]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [43]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.chromium.launch(headless=headless)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [44]:
driver, page = await open_browser()

In [45]:
url = 'https://www.change.org/'
await page.goto(url)

<Response url='https://www.change.org/' request=<Request url='https://www.change.org/' method='GET'>>

In [24]:
#not accept cookies
cookies = page.locator('//button[@data-testid="cookie-wall-reject"]')
await cookies.click()

### 2) Search bar und look for results in Germany

In [25]:
search = page.locator('//a[@data-qa="header-search-link" and @aria-label=""]')
await search.click()

In [26]:
germany = page.locator('//button[@aria-disabled="false"]//span[text() = "In Ihrer Nähe"]')
await germany.click()

### 3) Save results

In [28]:
source = await page.content()
with open(f'data/petition_change_org.html', 'w') as f:
    f.write(source)

### 3) Go through the pages and save results 

In [29]:
xpath_next = '//button[@aria-disabled="false" and @aria-label="Next button"]'

In [30]:
next_button = page.locator(xpath_next)

In [31]:
await expect(next_button).to_be_visible()

In [32]:
await next_button.click()

In [33]:
xpath_result = '//div[@aria-live="polite"]'

In [34]:
i = 1
collect = True

while collect:
    try:
        await expect(page.locator(xpath_next)).to_be_visible(timeout=1000)
        
        next_button = page.locator(xpath_next)
        await next_button.click()
        
        await page.locator(xpath_result).is_visible()
        
        source = await page.content()
        with open(f'data/petition_change_org_{i}.html', 'w') as f:
            f.write(source)
        
        i += 1
        
        await asyncio.sleep(2)
    
    except Exception as e:
        collect = False
        break

In [46]:
next_button = page.locator(xpath_next)
await next_button.click()

In [52]:
source = await page.content()
with open(f'data/petition_change_org_last.html', 'w') as f:
    f.write(source)

### 4) Parse the content

In [118]:
import glob
import pandas as pd
from bs4 import BeautifulSoup

In [134]:
files = glob.glob('data/trademark*.html')
len(files)

3368

In [135]:
xpath_petition = './/li[@class="ais-Hits-item w-full first:pt-0 pt-4 pb-4 md:pt-6 md:pb-6"]'

In [136]:
xpath_title = '//span[@data-variant="heading" and @data-size="small"]'

In [137]:
xpath_description = './/div[@class="hidden md:block"]//span[@data-variant="body"]'

In [138]:
xpath_date = './/p[contains(text(), "Am")]'

In [139]:
xpath_place = '//p[contains(text(), "Deutschland")]'

In [141]:
for fn in files:
    # read into a XML tree from a string
    tree = html.fromstring(open(fn).read())
    petitionen = tree.xpath(xpath_petition)

In [180]:
from lxml import html

data = []

for fn in files:

    # XML-Baum aus einer Datei lesen
    tree = html.fromstring(open(fn).read())

    # Alle Petitionselemente finden
    petitionen = tree.xpath(xpath_petition)  

    for petition in petitionen:
        # Titel extrahieren
        title = petition.xpath('.//span[@data-variant="heading" and @data-size="small"]/text()')
        title = title[0] if title else "Kein Titel gefunden"

        # Beschreibung extrahieren
        description = petition.xpath('.//div[@class="hidden md:block"]//span[@data-variant="body"]/text()')
        description = description[0] if description else "Keine Beschreibung gefunden"

        # Ort extrahieren
        place = petition.xpath('.//p[contains(text(), "Deutschland")]/text()')
        place = place[0] if place else "Kein Ort gefunden"

        signatures = petition.xpath('.//div[contains(@class, "text-typography-accent-primary")]//span[contains(text(), "Unterschriften")]/text()')
        signatures = signatures[0] if signatures else "Keine Unterschriften gefunden"

        # Startdatum relativ zum aktuellen <li> extrahieren
        start_date = petition.xpath('.//div[contains(@class, "flex max-w-full")]//span[contains(text(), "gestartet")]/text()')
        start_date = start_date[0] if start_date else "Kein Datum gefunden"

        # Daten sammeln
        data.append({
            "title": title,
            "description": description,
            "place": place,
            "signatures": signatures,
            "start_date": start_date
        })

In [181]:
df = pd.DataFrame(data)
df

,title,description,place,signatures,start_date
0,Verbot von Platzrunden und Sonntagsflügen von ...,Ich wohne direkt neben einem kleinem Sportflug...,Deutschland,53 Unterschriften,Am 29. Juli 2022 gestartet
1,Komplettausbau/-umstrukturierung des ÖPNV für ...,"Wir alle haben bestimmt schon den Fall erlebt,...",Deutschland,53 Unterschriften,Am 16. Mai 2022 gestartet
2,Unterstützt die zukunftsorientierte Entwicklun...,Vogelsberg im Aufbruch: Nachhaltigkeit und Mod...,"Vogelsberg, Deutschland",53 Unterschriften,Am 12. Dezember 2024 gestartet
3,Senken Sie die Geschwindigkeitsüberschreitunge...,Als Anwohner des Kerspeweges und Vater von dre...,"Kierspe, Deutschland",53 Unterschriften,Am 10. Dezember 2024 gestartet
4,Aufhebung des Böllerverbotes 2021/2022,"Das Jahr 2021 war eine Katastrophe, welches wi...",Deutschland,53 Unterschriften,Am 2. Dezember 2021 gestartet
...,...,...,...,...,...
26938,Wir fordern den selben Coronaschutz für Kinder...,Seit über einem Jahr ist der Regierung bekannt...,Deutschland,113 Unterschriften,Am 23. Mai 2021 gestartet
26939,Aufhebung des Verbotes teils auf gehwegen zu p...,Seit neusten werden in Mannheim Zettel an Auto...,Deutschland,113 Unterschriften,Am 27. September 2021 gestartet
26940,Aufenthaltsbestimmungsrecht für mein Sohn,Hallo meine Lieben ich kämpfe seit einiger Zei...,Deutschland,113 Unterschriften,Am 21. Januar 2019 gestartet
26941,"Kinder-, Radfahrer- und Fußgängerfreundliche A...",Die Fußgängerfeindliche Ampel am Ostbahnhof ka...,Deutschland,113 Unterschriften,Am 9. März 2022 gestartet


In [182]:
df.to_csv('petitionen.csv', index=False)